In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("/kaggle/input/bigbasket-entire-product-list-28k-datapoints/BigBasket Products.csv")
print(df.shape)
df.head()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna()   #droping null values
df.shape

In [ ]:
print(df.groupby(['product','brand']).size())  #so we have duplicates
df = df.drop_duplicates(subset=['product','brand'])  #droping duplicates
df.shape

# Exploratory Data Analysis

In [ ]:
chart=sns.countplot(x=df['category'])
chart.set_xticklabels(chart.get_xticklabels(), rotation=-90)

In [ ]:
chart=sns.countplot(x=df['type'])

In [ ]:
category_counts=df.groupby('category').agg({'sub_category': 'nunique', 'brand': 'nunique'})
category_counts

In [ ]:
brand_category_counts = df.groupby('brand').agg({'category': 'nunique'})
brand_category_counts
print("brands having products in one category",len(brand_category_counts[brand_category_counts['category']>1]))
print("brands having products in more than one category",len(brand_category_counts[brand_category_counts['category']==1]))

In [ ]:
products=df.groupby('type').agg({'category':'nunique','sub_category':'nunique','brand':'nunique','product':'nunique'})
products

In [ ]:
types=df.groupby('type').agg({'product':'nunique','brand':'nunique'})
types

### Observations:-
* Beauty & hygine is the most dominated category in big basket
* brands and categories are many-to-many relationship
* product names are unique, type is unique wrt category and subcategory
* Each type had multiple products and brands 
* So, type and discription alone can uniquely recommend a product but we need to consider category and sub category into consideration because if a user search for a product based on category or sub category and those exact key words may get missed in the description and type.


# Data Preprocessing

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
nltk.download('stopwords')
ps=PorterStemmer()
stopwords=stopwords.words('english')

In [ ]:
def pre_process(text):
    text=[i.lower() for i in text.split() if i.isalpha()]
    text=[ps.stem(i) for i in text if i not in stopwords]
    return ' '.join(text)

In [ ]:
# creating preprocessing data which is out training dataset
X=[]
for c1,c2,c3,c4,c5 in zip(df['product'],df['category'],df['sub_category'],df['type'],df['description']):
    combined_txt=c1+' '+c2+' '+c3+' '+c4+' '+c5
    result=pre_process(combined_txt)
    X.append(result)
print(X[:5])

In [ ]:
vector=TfidfVectorizer()
X=vector.fit_transform(X)
X=X.toarray()

In [ ]:
X[:5]
print(len(X),len(X[0]))

# Recommendation model

In [ ]:
 from sklearn.neighbors import NearestNeighbors

In [ ]:
model=NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(X)

In [ ]:
def predict(text):
    text=[pre_process(text)]
    res =vector.transform(text)
    res =res.toarray()
    pred = model.kneighbors(res)
    return df.iloc[pred[1][0]]

In [ ]:
predict('face wash')   #testing with product name directly

In [ ]:
predict('home utilities')   # testing with little complex name

In [ ]:
# testing with features of product (description type)
predict('I want a product which is used for cleaning my hair')

In [ ]:
predict('I want a product which is used for cleaning my hair which should be a ayurvedic product')

# Cluster Products Based On Description
lets assume In the dataset we don't had category, sub category and type of product and we had all the products with description. Now we try to categorize them using Kmeans clustering Algo

In [ ]:
X1=df['description'].apply(lambda x:pre_process(x))
print(X1[:2])

In [ ]:
vector1=TfidfVectorizer()
X1=vector.fit_transform(X1)
X1=X1.toarray()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10).fit(X1)

In [ ]:
classified_df=df.drop(["sale_price","market_price","rating"], axis=1)

In [ ]:
classified_df['new_category']=kmeans.labels_
classified_df

In [ ]:
classified_df.loc[(classified_df['new_category']==1)] #all edibles in cat 1

In [ ]:
classified_df.loc[(classified_df['new_category']==0)] #hair related as cat 0

In [ ]:
classified_df.loc[(classified_df['new_category']==2)]  #kitchen and household as cat2

In [ ]:
classified_df.loc[(classified_df['new_category']==3)]  #cookies and chocolates as cat3